# 1.5 「ファインチューニング」で精度向上を実現する方法  
- 本ファイルでは、学習済みのVGGモデルを使用し、ファインチューニングでアリとハチの画像を分類するモデルを学習します

# 学習目標  
1.PyTorchでGPUを使用する実装コードを書けるようになる  
2.最適化手法の設定において、層ごとに異なる学習率を設定したファインチューニングを実装できるようになる  
3.学習したネットワークを保存・ロードできるようになる 

In [ ]:
# パッケージのimport
import os
import numpy as np
import json
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import random


from tqdm import tqdm

import torch
import torchvision
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torchvision import models, transforms
from sklearn.model_selection import train_test_split 

In [ ]:
# PyTorchのバージョン確認
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

In [ ]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

# 入力画像の前処理クラスを作成

In [ ]:

resize = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_transforms = transforms.Compose([
                transforms.RandomResizedCrop(
                    resize, scale=(0.5, 1.0)),  # データオーギュメンテーション
                transforms.RandomHorizontalFlip(),  # データオーギュメンテーション
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
    ])
val_transforms = transforms.Compose([
                transforms.Resize(resize),  # リサイズ
                transforms.CenterCrop(resize),  # 画像中央をresize×resizeで切り取り
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
    ])

# テストデータを表示

In [ ]:
# 画像前処理の動作を確認

# 1. 画像読み込み
image_file_path = '../input/cassava-leaf-disease-classification/test_images/2216849948.jpg'
img = Image.open(image_file_path)  # [高さ][幅][色RGB]

# 2. 元の画像の表示
plt.imshow(img)
plt.show()

# 3. 画像の前処理と処理済み画像の表示
img_transformed = train_transforms(img)  # torch.Size([3, 224, 224])

# (色、高さ、幅)を (高さ、幅、色)に変換し、0-1に値を制限して表示
img_transformed = img_transformed.numpy().transpose((1, 2, 0))
img_transformed = np.clip(img_transformed, 0, 1)
plt.imshow(img_transformed)
plt.show()

# DataSetを作成

In [ ]:
PATH = "../input/cassava-leaf-disease-classification/train_images/"
IMG_SIZE = 224

class CassavaDataset(data.Dataset):
    def __init__(self,path,image_ids,labels,image_size, mode='val'):
        self.image_ids = image_ids
        self.labels = labels
        self.path = path
        self.image_size = image_size
        self.mode = mode

    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self,item):
      image_ids = str(self.image_ids[item])
      labels = self.labels[item]
      img = Image.open(self.path+image_ids)
      
      if self.mode=="train":
        return train_transforms(img),torch.tensor(labels,dtype=torch.long)
      else:
        return val_transforms(img),torch.tensor(labels,dtype=torch.long)
        #return torch.tensor(img,dtype=torch.float),torch.tensor(labels,dtype=torch.long)
    
dfx = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
xtrain, xval, ytrain, yval = train_test_split(dfx["image_id"].values,
                                              dfx.label.values,
                                              test_size = 0.1, random_state=0)
IMG_SIZE = 224

# 実行
train_dataset = CassavaDataset(PATH,xtrain,ytrain,IMG_SIZE, mode="train")
val_dataset   = CassavaDataset(PATH,xval,yval,IMG_SIZE)
print(dfx)

# DataLoaderを作成

In [ ]:
IMG_SIZE = 224

# DataLoaderを作成する
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

# ネットワークモデルの作成

In [ ]:
# 学習済みのVGG-16モデルをロード

# VGG-16モデルのインスタンスを生成
use_pretrained = True  # 学習済みのパラメータを使用
#use_pretrained = False  # 学習済みのパラメータを使用
net = models.vgg16(pretrained=use_pretrained)

# VGG16の最後の出力層の出力ユニットを病気4種と健康の5つに付け替える
net.classifier[6] = nn.Linear(in_features=4096, out_features=5)

# 訓練モードに設定
net.train()

print('ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました')

# 損失関数を定義
* 損失関数は、モデルがいかに問題を学習・推論できるかを図る指標”ロス(損失)”の計算方法を指定する

In [ ]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()

# 最適化手法を設定
* OptimizerがVGGモデルのどこのパラメータを更新(＝学習・最適化)するかを指定する
* Optimizerは”誤差逆伝搬法”を用いて、損失関数を最小化するようにVGGモデルのパラメータを更新(＝学習・最適化)役割を担う

In [ ]:
# ファインチューニングで学習させるパラメータを、変数params_to_updateの1～3に格納する

params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 学習させる層のパラメータ名を指定
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight",
                        "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

# パラメータごとに各リストに格納する
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1に格納：", name)

    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2に格納：", name)

    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3に格納：", name)

    else:
        param.requires_grad = False
        print("勾配計算なし。学習しない：", name)

In [ ]:
# 最適化手法の設定
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)

# 学習・検証を実施

In [ ]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # 初期設定
    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # epochのループ
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数

            # 未学習時の検証性能を確かめるため、epoch=0の訓練は省略
            if (epoch == 0) and (phase == 'train'):
                continue

            # データローダーからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloaders_dict[phase]):

                # GPUが使えるならGPUにデータを送る
                inputs = inputs.to(device)
                labels = labels.to(device)
                                
                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 損失を計算
                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 結果の計算
                    epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率を表示
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            

In [ ]:
# 学習・検証を実行する
num_epochs=1
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

# 学習したネットワークを保存・ロード

In [ ]:
# PyTorchのネットワークパラメータの保存
save_path = './weights_fine_tuning.pth'
torch.save(net.state_dict(), save_path)

In [ ]:
# PyTorchのネットワークパラメータのロード
load_path = './weights_fine_tuning.pth'
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

# GPU上で保存された重みをCPU上でロードする場合
load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})
net.load_state_dict(load_weights)

In [ ]:
from torch.utils.data import Dataset ,DataLoader

IMG_SIZE=224
TEST_FILE_PATH = "../input/cassava-leaf-disease-classification/test_images/"
    
class CassavaTestDataset(data.Dataset):
    def __init__(self,path,image_ids,image_size, mode='val'):
        print(path)
        print(image_ids)
        print(image_size)
        self.image_ids = image_ids
        self.path = path
        self.image_size = image_size

    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self,item):
      image_ids = str(self.image_ids[item])
      img = Image.open(self.path+image_ids)
    
      return val_transforms(img)
        
sample = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv")
test_dataset = CassavaTestDataset(TEST_FILE_PATH,sample.image_id,sample.label,IMG_SIZE)
test_loader = DataLoader(test_dataset,
                      batch_size=1,
                      shuffle=False)

# 初期設定
# GPUが使えるかを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

# ネットワークをGPUへ
net.to(device)
    
fin_outputs = []

for inputs in test_loader:

    # GPUが使えるならGPUにデータを送る
    inputs = inputs.to(device)
    
    outputs = net(inputs)
    outputs = nn.Softmax(dim=-1)(outputs)
    outputs = torch.argmax(outputs,dim=1)
    fin_outputs.append(outputs.cpu().detach().numpy())
                
sample["label"] = np.array(fin_outputs).reshape(-1)
sample[["image_id","label"]].to_csv("submission.csv",index=False)
sample.head()